In [1]:
import tensorflow as tf
from scipy import misc
import numpy as np
import sys

from load_dataset import load_batch
from ssim import MultiScaleSSIM
import models
import utils
import vgg

In [2]:
# Defining size of the training image patches
PATCH_WIDTH = 100
PATCH_HEIGHT = 100
IMAGE_SIZE = PATCH_WIDTH * PATCH_HEIGHT * 3

# Model Paramters
batch_size = 12
train_size = None
learning_rate = 5e-4

# Loss weighting
w_content = 1
w_color = 0.005
w_texture = 0.005
w_tv = 10
data_dir = 'D:\\Study\\DL\\Project\\Biom_project\\GAN\\updated_source\\dped\\dped\\dped\\sony\\training_data'
test_data_dir = 'D:\\Study\\DL\\Project\\Biom_project\\GAN\\updated_source\\dped\\dped\\dped\\sony\\test_data\\patches'
vgg_dir = 'vgg_pretrained/imagenet-vgg-verydeep-19.mat'

#np.random.seed(50)

In [7]:
# Loading training and test data
train_data_Phone, train_data_Dslr = load_batch(data_dir, IMAGE_SIZE, 1000)
test_data_Phone, test_data_Dslr = load_batch(test_data_dir, IMAGE_SIZE, 100)

Starting train load start!!
Starting train load start!!


In [8]:
train_size = train_data_Phone.shape[0]
num_train_iters =  int(train_size / batch_size) #number of training iterations

print(num_train_iters)

eval_step = 200 # Perform evaluation after this many steps
TEST_SIZE = test_data_Phone.shape[0]

# Defining GPU Memory fraction
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.75)

83


In [16]:
#Gaussian noise added to stabilize training and prevent discriminator from overfitting
def gaussian_noise_layer(input_layer, std):
    noise = tf.random_normal(shape=tf.shape(input_layer), mean=0.0, stddev=std, dtype=tf.float32) 
    return input_layer + noise

# Defining computation graph
with tf.Graph().as_default(), tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    
    # Placeholders for training data
    Phone_ = tf.placeholder(tf.float32, [None, IMAGE_SIZE])
    Phone_image = tf.reshape(Phone_, [-1, PATCH_HEIGHT, PATCH_WIDTH, 3])
    Dslr_ = tf.placeholder(tf.float32, [None, IMAGE_SIZE])
    Dslr_image = tf.reshape(Dslr_, [-1, PATCH_HEIGHT, PATCH_WIDTH, 3])
    adv_ = tf.placeholder(tf.float32, [None, 1])

    # get processed enhanced image
    enhanced = models.resnet(Phone_image)
    reconstructed_phone = models.resnet(enhanced) #(From enhanced)
    
    # transform both Dslr and enhanced images to grayscale
    enhanced_gray = tf.reshape(tf.image.rgb_to_grayscale(enhanced), [-1, PATCH_WIDTH * PATCH_HEIGHT])
    Dslr_gray = tf.reshape(tf.image.rgb_to_grayscale(Dslr_image),[-1, PATCH_WIDTH * PATCH_HEIGHT])

    #Blur the Dslr and enhanced image
    enhanced_blur = tf.reshape(utils.blur(enhanced), [-1, PATCH_WIDTH * PATCH_HEIGHT * 3])
    Dslr_blur = tf.reshape(utils.blur(Dslr_image), [-1, PATCH_WIDTH * PATCH_HEIGHT * 3])
    
    #Add noise
    enhanced_blur = gaussian_noise_layer(enhanced_blur, .2)
    Dslr_blur = gaussian_noise_layer(Dslr_blur, .2)

    # push randomly the enhanced or Dslr image to an adversarial CNN-discriminator
    color_adversarial_ = tf.multiply(enhanced_blur, 1 - adv_) + tf.multiply(Dslr_blur, adv_)
    color_adversarial_image = tf.reshape(color_adversarial_, [-1, PATCH_HEIGHT, PATCH_WIDTH, 3])
    color_discrim_predictions = models.adversarial_color(color_adversarial_image)

    # push randomly the enhanced or Dslr image to an adversarial CNN-discriminator
    adversarial_ = tf.multiply(enhanced_gray, 1 - adv_) + tf.multiply(Dslr_gray, adv_)
    adversarial_image = tf.reshape(adversarial_, [-1, PATCH_HEIGHT, PATCH_WIDTH, 1])
    discrim_predictions = models.adversarial(adversarial_image)

    # losses
    # 1) texture (adversarial) loss
    
    discrim_target = tf.concat([adv_, 1 - adv_], 1)
    loss_discrim = - tf.reduce_sum(discrim_target * tf.log(tf.clip_by_value(discrim_predictions, 1e-10, 1.0)))
    loss_texture = - loss_discrim
    correct_predictions = tf.equal(tf.argmax(discrim_predictions, 1), tf.argmax(discrim_target, 1))
    discrim_accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
    

    # 2) color (adversarial) loss
    color_discrim_target = tf.concat([adv_, 1 - adv_], 1)
    loss_discrim_color = - tf.reduce_sum(color_discrim_target * tf.log(tf.clip_by_value(color_discrim_predictions, 1e-10, 1.0)))
    loss_color = - loss_discrim_color
    correct_predictions_color = tf.equal(tf.argmax(color_discrim_predictions, 1), tf.argmax(color_discrim_target, 1))
    color_discrim_accuracy = tf.reduce_mean(tf.cast(correct_predictions_color, tf.float32))
    

    # 3) content loss
    CONTENT_LAYER = 'relu5_4'
    phone_vgg = vgg.net(vgg_dir, vgg.preprocess(Phone_image * 255))  # Obtained content from enhanced VGG
    reconstructed_vgg = vgg.net(vgg_dir, vgg.preprocess(reconstructed_phone * 255))  # Obtained content from Input VGG
    content_size = utils._tensor_size(reconstructed_vgg[CONTENT_LAYER]) * batch_size
    loss_content = 2 * tf.nn.l2_loss(phone_vgg[CONTENT_LAYER] - reconstructed_vgg[CONTENT_LAYER]) / content_size

    # 4) total variation loss
    batch_shape = (batch_size, PATCH_WIDTH, PATCH_HEIGHT, 3)
    tv_y_size = utils._tensor_size(enhanced[:,1:,:,:])
    tv_x_size = utils._tensor_size(enhanced[:,:,1:,:])
    y_tv = tf.nn.l2_loss(enhanced[:,1:,:,:] - enhanced[:,:batch_shape[1]-1,:,:])
    x_tv = tf.nn.l2_loss(enhanced[:,:,1:,:] - enhanced[:,:,:batch_shape[2]-1,:])
    loss_tv = 2 * (x_tv/tv_x_size + y_tv/tv_y_size) / batch_size

    # final loss
    loss_generator = w_content * loss_content + w_tv * loss_tv + w_color * loss_color + w_texture * loss_texture
    

    # psnr loss
    #enhanced_flat = tf.reshape(enhanced, [-1, PATCH_SIZE])
    #loss_mse = tf.reduce_sum(tf.pow(Dslr_ - enhanced_flat, 2))/(PATCH_SIZE * batch_size)
    #loss_psnr = 20 * utils.log10(1.0 / tf.sqrt(loss_mse))

    # optimize parameters of image enhancement (generator) and discriminator networks
    generator_vars = [v for v in tf.global_variables() if v.name.startswith("generator")]
    discriminator_vars = [v for v in tf.global_variables() if v.name.startswith("discriminator")]
    color_discriminator_vars = [v for v in tf.global_variables() if v.name.startswith("discriminator_color")]

    train_step_gen = tf.train.AdamOptimizer(learning_rate).minimize(loss_generator, var_list=generator_vars)
    train_step_disc_color = tf.train.AdamOptimizer(learning_rate).minimize(loss_discrim_color, var_list=color_discriminator_vars)
    train_step_disc = tf.train.AdamOptimizer(learning_rate).minimize(loss_discrim, var_list=discriminator_vars)

    saver = tf.train.Saver(var_list=generator_vars, max_to_keep=100)

    print('Initializing variables')
    sess.run(tf.global_variables_initializer())

    print('Training network')

    train_loss_gen = 0.0
    train_acc_discrim = 0.0
    train_acc_discrim_color = 0.0

    all_zeros = np.reshape(np.zeros((batch_size, 1)), [batch_size, 1])
    test_crops_phone = test_data_Phone[np.random.randint(0, TEST_SIZE, batch_size), :]
    test_crops_dslr = test_data_Dslr[np.random.randint(0, TEST_SIZE, batch_size), :]
    
    logs = open('models/' + 'mdl.txt', "w+")
    logs.close()
    
    
    i = 0
    iteration_count = 0
    while(i < num_train_iters):
        
        #Train generator
        idx_train = np.random.randint(0, train_size, batch_size)
        Phone_images = train_data_Phone[idx_train]
        Dslr_images = train_data_Dslr[idx_train]
        [loss_temp, temp] = sess.run([loss_generator, train_step_gen], 
                                     feed_dict={Phone_: Phone_images, Dslr_: Dslr_images, adv_: all_zeros})

        print('Iteration:',iteration_count,' Gen_loss:',loss_temp)
        train_loss_gen += loss_temp / eval_step
        

        # train discriminator
        idx_train = np.random.randint(0, train_size, batch_size)
        
        # generate image swaps (Dslr or enhanced) for discriminator
        
        #if(iteration_count % 2 == 0): #Some papers suggest using same class images in a single batch
            #swaps = np.reshape(np.ones(batch_size, dtype=int),[batch_size,1])
        #else:
            #swaps = np.reshape(np.zeros(batch_size, dtype=int),[batch_size,1])
        
        swaps = np.reshape(np.random.randint(0, 2, batch_size), [batch_size, 1])
            
        Phone_images = train_data_Phone[idx_train]
        Dslr_images = train_data_Dslr[idx_train]
        
        [accuracy_temp, accuracy_temp_color, t_loss, c_loss] = sess.run([discrim_accuracy, color_discrim_accuracy, loss_texture, loss_color],
                                        feed_dict={Phone_: Phone_images, Dslr_: Dslr_images, adv_: swaps})

        #[accuracy_temp_color, c_loss, temp_color] = sess.run([, color_loss, train_step_disc_color],
         #                              feed_dict={Phone_: Phone_images, Dslr_: Dslr_images, adv_: swaps})


        print('Iteration:',iteration_count, ' Color loss:',c_loss, ' texture:', t_loss)
        
        train_acc_discrim += accuracy_temp / eval_step
        train_acc_discrim_color += accuracy_temp_color / eval_step
        
        #Increase the iteration
        i+=1
        iteration_count+=1
        
        #Evaluation
        if(iteration_count % eval_step == 0):
            
            #Loss and Accuracy
            print('Generator_loss:',train_loss_gen)
            print('Texture Discriminator Acc:',train_acc_discrim)
            print('Color Discriminator Acc:',train_acc_discrim_color)
            
            #Testing the model
            num_test_batches = int(test_data_Phone.shape[0] / batch_size)
            
            test_losses_gen = np.zeros((1, 4))
            test_accuracy_disc = 0.0
            #test_accuracy_disc_color = 0.0
            loss_ssim = 0.0
            
            for j in range(num_test_batches):

                be = j * batch_size
                en = (j+1) * batch_size

                swaps = np.reshape(np.random.randint(0, 2, batch_size), [batch_size, 1])

                Phone_images = test_data_Phone[be:en]
                Dslr_images = test_data_Dslr[be:en]
                
                [enhanced_crops, accuracy_disc, losses] = sess.run([enhanced, discrim_accuracy,
                                                                                       [loss_generator, loss_content, loss_color, loss_tv]],
                                                                                       feed_dict={Phone_: Phone_images, Dslr_: Dslr_images, adv_: swaps})

                test_losses_gen += np.asarray(losses) / num_test_batches
                test_accuracy_disc += accuracy_disc / num_test_batches
                #test_accuracy_disc_color += accuracy_disc_color / num_test_batches

                loss_ssim += MultiScaleSSIM(np.reshape(Phone_images * 255, [batch_size, PATCH_HEIGHT, PATCH_WIDTH, 3]),
                                                    enhanced_crops * 255) / num_test_batches

            #logs_disc = "step %d, | discriminator accuracy | train: %.4g, test: %.4g" % \
             #     (i, train_acc_discrim, test_accuracy_disc)
            
            logs_disc_color = "step %d, | discriminator accuracy | train: %.4g, test: %.4g" % \
                  (i, train_acc_discrim, test_accuracy_disc)

            logs_gen = "generator losses | train: %.4g, test: %.4g | content: %.4g, color: %.4g, tv: %.4g | ssim: %.4g\n" % \
                  (train_loss_gen, test_losses_gen[0][0], test_losses_gen[0][1], test_losses_gen[0][2],
                   test_losses_gen[0][3],  loss_ssim)

            #print(logs_disc)
            print(logs_disc_color)
            print(logs_gen)

            # save the results to log file

            logs = open('models/' + 'mdl.txt', "a")
            logs.write(logs_disc_color)
            logs.write('\n')
            logs.write(logs_gen)
            logs.write('\n')
            logs.close()

            # save visual results for several test image crops

            enhanced_crops = sess.run(enhanced, feed_dict={Phone_: test_crops_phone, Dslr_: test_crops_dslr, adv_: all_zeros})

            idx = 0
            for crop in enhanced_crops:
                before_after = np.hstack((np.reshape(test_crops_phone[idx], [PATCH_HEIGHT, PATCH_WIDTH, 3]), crop))
                misc.imsave('results/' + str(idx) + '_iteration_' + str(iteration_count) + '.jpg', before_after)
                idx += 1

            # save the model that corresponds to the current iteration
            saver.save(sess, 'models/' + '_iteration_' + str(i) + '.ckpt', write_meta_graph=False)
            
            train_loss_gen = 0.0
            train_acc_discrim = 0.0
            #train_acc_discrim_color = 0.0
        
        if(i>=num_train_iters):            
            del train_data_Phone
            del train_data_Dslr 
            
            # Loading training
            i = 0
            train_data_Phone, train_data_Dslr = load_batch(data_dir, IMAGE_SIZE, 1000)
            train_size = train_data_Phone.shape[0]
            num_train_iters = int(train_size / batch_size)
            
        

loss_generator: ()
loss_content: ()
Initializing variables
Training network
Iteration: 0  Gen_loss: 13.028535
Iteration: 0  Color loss: -8.998259  texture: -7.7399693
Iteration: 1  Gen_loss: 15.288154
Iteration: 1  Color loss: -8.467376  texture: -8.277551
Iteration: 2  Gen_loss: 9.997047
Iteration: 2  Color loss: -8.45497  texture: -8.5123205
Iteration: 3  Gen_loss: 10.571912
Iteration: 3  Color loss: -9.18135  texture: -8.518494
Iteration: 4  Gen_loss: 8.540189
Iteration: 4  Color loss: -8.526962  texture: -8.54395
Iteration: 5  Gen_loss: 9.166912
Iteration: 5  Color loss: -9.517251  texture: -8.368221
Iteration: 6  Gen_loss: 12.00329
Iteration: 6  Color loss: -9.585125  texture: -8.919539
Iteration: 7  Gen_loss: 12.230286
Iteration: 7  Color loss: -8.598246  texture: -8.299528
Iteration: 8  Gen_loss: 11.70713
Iteration: 8  Color loss: -9.515347  texture: -8.435669
Iteration: 9  Gen_loss: 14.755626
Iteration: 9  Color loss: -8.435253  texture: -8.9839
Iteration: 10  Gen_loss: 15.1836

Iteration: 89  Gen_loss: 10.804993
Iteration: 89  Color loss: -8.920996  texture: -9.090421
Iteration: 90  Gen_loss: 5.903977
Iteration: 90  Color loss: -9.518967  texture: -8.895313
Iteration: 91  Gen_loss: 19.440939
Iteration: 91  Color loss: -8.450764  texture: -8.384415
Iteration: 92  Gen_loss: 15.077523
Iteration: 92  Color loss: -9.408008  texture: -8.455665
Iteration: 93  Gen_loss: 9.09483
Iteration: 93  Color loss: -9.305752  texture: -8.597868
Iteration: 94  Gen_loss: 10.743551
Iteration: 94  Color loss: -9.184122  texture: -8.375246
Iteration: 95  Gen_loss: 8.453943
Iteration: 95  Color loss: -9.646286  texture: -8.508383
Iteration: 96  Gen_loss: 12.721695
Iteration: 96  Color loss: -8.577924  texture: -8.292318
Iteration: 97  Gen_loss: 11.263803
Iteration: 97  Color loss: -9.238813  texture: -8.390581
Iteration: 98  Gen_loss: 9.707894
Iteration: 98  Color loss: -8.446655  texture: -8.890578
Iteration: 99  Gen_loss: 13.163615
Iteration: 99  Color loss: -9.21212  texture: -8.5

Iteration: 177  Gen_loss: 7.840609
Iteration: 177  Color loss: -8.738543  texture: -8.987914
Iteration: 178  Gen_loss: 12.044075
Iteration: 178  Color loss: -9.526526  texture: -8.738125
Iteration: 179  Gen_loss: 11.7564335
Iteration: 179  Color loss: -9.106801  texture: -8.400782
Iteration: 180  Gen_loss: 9.5157175
Iteration: 180  Color loss: -8.834691  texture: -8.488715
Iteration: 181  Gen_loss: 6.9975786
Iteration: 181  Color loss: -9.814733  texture: -8.399469
Iteration: 182  Gen_loss: 15.700876
Iteration: 182  Color loss: -9.214581  texture: -8.451211
Iteration: 183  Gen_loss: 10.872897
Iteration: 183  Color loss: -9.29407  texture: -8.80826
Iteration: 184  Gen_loss: 6.9057837
Iteration: 184  Color loss: -9.2290535  texture: -8.752427
Iteration: 185  Gen_loss: 10.901352
Iteration: 185  Color loss: -8.954287  texture: -9.213974
Iteration: 186  Gen_loss: 7.761892
Iteration: 186  Color loss: -8.530426  texture: -8.688229
Iteration: 187  Gen_loss: 15.38907
Iteration: 187  Color loss:

D:\Anaconda\envs\dl\lib\site-packages\ipykernel_launcher.py:246: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Iteration: 200  Gen_loss: 11.1669035
Iteration: 200  Color loss: -8.8510275  texture: -8.435242
Iteration: 201  Gen_loss: 9.477606
Iteration: 201  Color loss: -8.400403  texture: -8.579412
Iteration: 202  Gen_loss: 6.667566
Iteration: 202  Color loss: -9.009453  texture: -8.7441
Iteration: 203  Gen_loss: 7.699119
Iteration: 203  Color loss: -8.832666  texture: -8.520831
Iteration: 204  Gen_loss: 9.793814
Iteration: 204  Color loss: -8.915467  texture: -8.181386
Iteration: 205  Gen_loss: 8.419315
Iteration: 205  Color loss: -10.095369  texture: -8.246922
Iteration: 206  Gen_loss: 6.3270936
Iteration: 206  Color loss: -9.565374  texture: -8.528383
Iteration: 207  Gen_loss: 7.185551
Iteration: 207  Color loss: -9.062829  texture: -8.414318
Iteration: 208  Gen_loss: 8.858623
Iteration: 208  Color loss: -8.864526  texture: -8.399282
Iteration: 209  Gen_loss: 5.4373126
Iteration: 209  Color loss: -8.835346  texture: -8.992813
Iteration: 210  Gen_loss: 7.5796866
Iteration: 210  Color loss: -9

...

Iteration: 3694  Gen_loss: 2.2954059
Iteration: 3694  Color loss: -8.777945  texture: -8.743539
Iteration: 3695  Gen_loss: 4.1538854
Iteration: 3695  Color loss: -8.718966  texture: -8.936272
Iteration: 3696  Gen_loss: 3.338663
Iteration: 3696  Color loss: -8.892504  texture: -8.797722
Iteration: 3697  Gen_loss: 2.4980347
Iteration: 3697  Color loss: -9.438906  texture: -8.441299
Iteration: 3698  Gen_loss: 4.5085444
Iteration: 3698  Color loss: -8.213527  texture: -8.710943
Iteration: 3699  Gen_loss: 6.4062705
Iteration: 3699  Color loss: -8.519379  texture: -8.717611
Iteration: 3700  Gen_loss: 3.440693
Iteration: 3700  Color loss: -9.822372  texture: -8.366482
Iteration: 3701  Gen_loss: 4.3819623
Iteration: 3701  Color loss: -8.735029  texture: -8.589506
Iteration: 3702  Gen_loss: 3.0985696
Iteration: 3702  Color loss: -8.940187  texture: -8.455603
Iteration: 3703  Gen_loss: 3.841934
Iteration: 3703  Color loss: -8.77596  texture: -8.646637
Iteration: 3704  Gen_loss: 5.0467944
Iterati

Iteration: 3780  Gen_loss: 2.8285654
Iteration: 3780  Color loss: -8.735293  texture: -8.543992
Iteration: 3781  Gen_loss: 3.8769212
Iteration: 3781  Color loss: -8.886002  texture: -8.592044
Iteration: 3782  Gen_loss: 4.8748
Iteration: 3782  Color loss: -8.612627  texture: -8.830132
Iteration: 3783  Gen_loss: 3.1822217
Iteration: 3783  Color loss: -8.85032  texture: -8.622922
Iteration: 3784  Gen_loss: 2.81998
Iteration: 3784  Color loss: -8.697956  texture: -8.562006
Iteration: 3785  Gen_loss: 2.884764
Iteration: 3785  Color loss: -8.697449  texture: -8.67649
Iteration: 3786  Gen_loss: 3.6788769
Iteration: 3786  Color loss: -8.577078  texture: -8.57541
Iteration: 3787  Gen_loss: 3.6019359
Iteration: 3787  Color loss: -9.505721  texture: -8.826348
Iteration: 3788  Gen_loss: 3.352225
Iteration: 3788  Color loss: -9.232831  texture: -8.613613
Iteration: 3789  Gen_loss: 4.9785743
Iteration: 3789  Color loss: -8.731916  texture: -8.261224
Iteration: 3790  Gen_loss: 6.6955867
Iteration: 37

Iteration: 3863  Gen_loss: 3.3710592
Iteration: 3863  Color loss: -9.042251  texture: -9.188271
Iteration: 3864  Gen_loss: 5.195541
Iteration: 3864  Color loss: -8.585554  texture: -8.682263
Iteration: 3865  Gen_loss: 2.6839066
Iteration: 3865  Color loss: -8.79497  texture: -8.691559
Iteration: 3866  Gen_loss: 5.571335
Iteration: 3866  Color loss: -8.549753  texture: -8.9670925
Iteration: 3867  Gen_loss: 4.051592
Iteration: 3867  Color loss: -8.66101  texture: -8.850464
Iteration: 3868  Gen_loss: 3.6243155
Iteration: 3868  Color loss: -8.290936  texture: -8.73026
Iteration: 3869  Gen_loss: 2.5710409
Iteration: 3869  Color loss: -8.726801  texture: -8.393053
Iteration: 3870  Gen_loss: 3.202574
Iteration: 3870  Color loss: -8.676095  texture: -8.335441
Iteration: 3871  Gen_loss: 4.1127524
Iteration: 3871  Color loss: -8.901681  texture: -8.726923
Iteration: 3872  Gen_loss: 3.9628212
Iteration: 3872  Color loss: -8.924904  texture: -8.9985695
Iteration: 3873  Gen_loss: 7.0541477
Iteratio

Iteration: 3949  Gen_loss: 5.290916
Iteration: 3949  Color loss: -9.210446  texture: -8.616201
Iteration: 3950  Gen_loss: 4.1281776
Iteration: 3950  Color loss: -8.564899  texture: -8.287037
Iteration: 3951  Gen_loss: 2.6775403
Iteration: 3951  Color loss: -8.553145  texture: -8.377535
Iteration: 3952  Gen_loss: 6.019726
Iteration: 3952  Color loss: -8.396665  texture: -8.481253
Iteration: 3953  Gen_loss: 2.8282437
Iteration: 3953  Color loss: -8.285299  texture: -8.213072
Iteration: 3954  Gen_loss: 5.2551312
Iteration: 3954  Color loss: -9.42879  texture: -8.419703
Iteration: 3955  Gen_loss: 2.6867242
Iteration: 3955  Color loss: -9.484802  texture: -8.915298
Iteration: 3956  Gen_loss: 2.4124942
Iteration: 3956  Color loss: -8.504835  texture: -8.823189
Iteration: 3957  Gen_loss: 3.8694499
Iteration: 3957  Color loss: -9.092168  texture: -8.869648
Iteration: 3958  Gen_loss: 3.7675364
Iteration: 3958  Color loss: -8.861155  texture: -8.262308
Iteration: 3959  Gen_loss: 2.8435507
Iterat

KeyboardInterrupt: 